In [8]:
import pandas as pd
import openai
import csv
import numpy as np
from time import sleep
import json
import ast
import re

openai.api_key ="API_KEY"

def get_embedding(text, model="text-embedding-ada-002"):
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

user_text = "졸업요건이 뭐야?"

Section = """졸업
제34조 (졸업 사정) 제1항, 졸업 대상자는 다음 각호의 졸업요건을 충족하여야 한다: 총 이수학점 130학점 이상 취득, 교양과목 영역별 지정 이수학점 취득, 교양필수, 공필수과목의 전 학점 취득, 주전공의 기본전공과목 45학점 이상 취득, 5. 부전공, 복수전공 또는 연계·융합 전공 중 하나를 이수하거나 주전공의 심화 전공과목을 24학점 이상 추가 취득, 전 학년 성적 평점 평균이 2.00 이상, 졸업 인증제 통과(북한이탈주민, 학군 제휴 군위탁생, 계약학과 학생 제외), 해당 학부(과)별 운영내규에 의거 요구되는 졸업요건을 충족, 4번에도 불구하고 자유전공학부 졸업대상자의 전공과목 취득 학점은 별도로 정할 수 있다.
10. 편입학생은 전 4호 및 전 5호의 졸업요건 대신 주전공을 57학점 이상 취득하여야 한다.
제2항, 전 1항에 따른 졸업요건을 충족하지 못하여 졸업이 불가능한 자가 소정의 등록금을 납부하고 졸업요건을 충족하면 그 학년도에 졸업할 수 있다. 단, 전 1항 제7호의 졸업요건만 충족하지 못한 경우에는 등록금을 납부하지 않는다. 제3항, 부전공 이수 인정은 전 1항의 조건을 갖춘 자로서 제24조 제1항 제6호에 따른다. 제4항, 복수전공 이수 인정은 전 1항의 조건을 갖춘 자로서 제25조 제1항 제6호에 따른다. 제5항, 연계·융합 전공 이수 인정은 전 1항의 조건을 갖춘 자로서 제25조의3 제2항 제4호에 따른다. 제6항, 국제항공 교육 인증프로그램에 참여하는 자는 학칙 제26조에 의거 해당 학부(과)별 국제항공 교육 인증 프로그램 내규의 졸업 이수학점을 충족하여야 한다.
제34조의2 (조기졸업) 제1항, 조기졸업의 신청 시기는 2학년 수료 후 학기 초 조기졸업 신청서를 소속 학부(과)장의 허가를 받아 교무처에 제출하여야 한다. 단, 조기졸업 이수 신청이 허가된 자는 상위 학년에 전공필수 과목의 수강 신청을 할 수 있다. 
제2항, 조기 졸업자는 본 대학에서 6개 학기 이상 등록을 필하고 졸업소요학점을 취득하여야 하며, 전 학년 통산 평점 평균이 4.0 이상이어야 한다. 제3항, 재학 중 1회 이상 학사경고를 받았을 때는 조기졸업 대상에서 제외된다. 제4항, 전 1항의 단서 사항에 의거 상위학년의 필수 과목을 이수한 이후 교과과정 변경으로 필수과목이 변경된 경우에도 필수과목을 이수한 것으로 본다. 
제34조의3 (졸업 인증제) 제1항, 제34조 제1항 제6호에 의한 졸업 인증 선택 분야는 다음 각호
와 같다.
1. 글로벌 소통 역량 인증
2. 전문 탐구 역량 인증
3. 통섭 융합 역량 인증
4. 창의 도전역량 인증
5. 윤리 인성 역량 인증
제2항, 졸업대상자는 제1항의 졸업 인증 선택 분야 중 글로벌 소통 역량을 포함하여 2개 이상의 영역에서 인증을 받아야 하며, 졸업 인증에 관한 세부 사항은 별도로 정한다. 제3항, 북한이탈주민, 학군 제휴 군위탁생, 계약학과 학생은 예외로 한다. 제4항, 전 3항 외 예외로 인정하는 사유가 있는 경우 총장의 승인을 받아 면제할 수 있다. 제5항, 졸업 인증을 통과하지 못한 자에게는 해당 전공 학사학위를 수여하지 않고 수료하거나, 일정한 요건을 갖추어 재학 상태를 유지할 수 있다. 단, 수료 이후에 이를 필할 경우에는 해당 학기 말에 학사학위를 수여한다.
제34조의4 (우수졸업생) 학칙 제28조 졸업 자격을 충족한 학생 중 학업성적이 뛰어나고 품행이 단정한 학생에게는 다음 각호와 같이 구분하여 학적부 및 학위증 등에 기재한다. 
1. 최우수(Summa Cum Laude): 누적 평점 평균이 4.25/4.50 이상이거나 학부(과)별 상위 2% 이내인 자
2. 우수(Magna Cum Laude): 누적 평점 평균이 4.00/4.50 이상인 자
3. 우등(Cum Laude): 누적 평점 평균이 3.75/4.50 이상인 자
제34조의5 (학위증 및 수여 학위) 제1항, 학위증에는 소속 학부(과)와 주전공, 부·복수전공, 연계·융합 전공, 심화전공 등의 전공 이수에 관한 사항을 기재한다. 제2항, 학위증 서식 및 전공별 수여 학위명은 각각 및 과 같다. 제3항, 국제항공 교육 인증프로그램의 전공명은 과 같다."""

Section2 = """졸업요건

전공 졸업요건

학부(과) : 소프트웨어학과
학번 구분 : 2012학번까지
학칙/학칙 시행세칙 - 졸업학점 : 140, 편입생 졸업학점 : 70
학과운영내규 전공학점(전필+전선) : 60학점 (4학년에 개설된 전공선택 2과목 이상 이수/단,인턴쉽 과목은 1과목으로만 인정함)
비고 : 전 학년 평점 평균 2.00 이상

학부(과) : 소프트웨어학과
학번 구분 : 2013학번 이후 ~ 2018학번 까지
학칙/학칙 시행세칙 - 졸업학점 : 130, 편입생 졸업학점 : 65

학부(과) : 소프트웨어학과
학번 구분 : 2014학번 까지
학칙/학칙 시행세칙 - 졸업인증제 기준 점수 : 600점

학부(과) : 소프트웨어학과
학번 구분 : 2015학번 이후
학칙/학칙 시행세칙 - 졸업인증제 기준 점수 : 650점

2019학번부터 적용 : 주전공(기본)이수 학점을 반드시 취득해야하며, 교과과정의 전공필수 교과목은 반드시 이수해야 한다. 단일(심화)전공 및 다전공 중 1개의 전공을 선택하여 아래표와 같이 전공학점을 취득해야한다.

주전공 : 기본전공 45학점, 심화전공 24학점 이상, 복수전공 42학점 이상, 부전공 24학점 이상, 연계융합전공 36학점 이상
단일전공 : 69학점
다전공 : 87학점
복수전공 : 69학점
연계융합전공 : 81학점

AI융합전공 안내 : 한국항공대학교 AI융합전공 (kau.ac.kr)
교양 졸업이수학점은 아래 링크를 참고하기 바람 : http://college.kau.ac.kr/web/pages/gc51336h.do"""

user_text_vec = get_embedding(user_text, model='text-embedding-ada-002')
Section_vec = get_embedding(Section, model='text-embedding-ada-002')
Section_vec2 = get_embedding(Section2, model='text-embedding-ada-002')

score = np.dot(np.array(user_text_vec), np.array(Section_vec))
print(score)

score2 = np.dot(np.array(user_text_vec), np.array(Section_vec2))
print(score2)

0.8553275095116267
0.8303006242084372


In [10]:
def translate_func(temp_manual):
    sys = f"""function_name: [trans_sentence]
    input: ["Korean_sentence"]
    output: ["English_sentence"]
    rule: [I want you to act as an English translator. I will provide you with input forms including "sentence" in any language and you will detect the language, translate it, in English.]

    # Start translate!
    input : [trans_sentence({temp_manual})]"""
    completion = openai.ChatCompletion.create( 
          model="gpt-3.5-turbo-1106",  
#           model="gpt-4-1106-preview",  
          messages=[
          {"role": "system", "content": f'"{sys}"'},
    #       {"role": "user", "content": f'"{user}"'},
          ], temperature =  0.0001)
    answer = completion['choices'][0]['message']['content']
    return answer 

user_text = "What are the graduation requirements?"
Section_eng = translate_func(Section)
Section_eng2 = translate_func(Section2)

user_text_vec = get_embedding(user_text, model='text-embedding-ada-002')
Section_vec = get_embedding(Section_eng, model='text-embedding-ada-002')
Section_vec2 = get_embedding(Section_eng2, model='text-embedding-ada-002')

score = np.dot(np.array(user_text_vec), np.array(Section_vec))
print(score)

score2 = np.dot(np.array(user_text_vec), np.array(Section_vec2))
print(score2)

0.8656140564939834
0.8378749858206243


In [12]:
print(Section_eng)

output: ["Article 34 (Graduation) Paragraph 1, Graduates must satisfy the following graduation requirements: acquire 130 or more total credits, acquire designated credits for general education subjects, acquire credits for general education requirements and mandatory subjects, acquire 45 or more credits for the major subjects, complete a minor, double major, or integrated major, or acquire 24 or more additional credits for advanced major subjects, maintain a GPA of 2.00 or higher, pass the graduation certification (excluding North Korean defectors, students under school-affiliated military consignment, and students in contracted departments), and satisfy the graduation requirements specified by the respective department's regulations, and the credits for major subjects of graduates from the School of Liberal Arts and Sciences can be separately determined regardless of the above. 
10. Transfer students must acquire 57 or more credits for the major instead of the graduation requirements 

In [23]:
import tiktoken
enc = tiktoken.get_encoding('cl100k_base')
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo-1106")

Section = """졸업요건이 뭐야?"""
Section_eng = """What are the graduation requirements?"""

tokens = tokenizer.encode(Section)
print(len(tokens))
tokens = tokenizer.encode(Section_eng)
print(len(tokens))
# tokenizer.decode(tokens)

13
6
